In [20]:
import numpy as np
import skimage as sk
from glob import glob
import napari
from cellpose import io, models, train
import os


In [21]:
img_files = sorted(glob("../Microglia_CellPose_Model/Images/*.tif"))
mask_files = sorted(glob("../Microglia_CellPose_Model/Masks/*.tif"))
imgs = list(map(sk.io.imread, img_files))
masks = list(map(sk.io.imread, mask_files))

In [ ]:
os.mkdir('../Microglia_CellPose_Model/Training')
save = '../Microglia_CellPose_Model/Training'
for i in range(len(imgs)):
    split_img = np.split(imgs[i], imgs[i].shape[0],axis=0)
    split_mask = np.split(masks[i], masks[i].shape[0],axis=0)
    for c in range(len(split_img)):
        if np.max(split_mask[c]) == 0:
            continue
        else:
        sk.io.imsave(os.path.join(save,"Img_0"+str(i)+"_"+str(c)+".tif"),split_img[c])
        sk.io.imsave(os.path.join(save,"Img_0"+str(i)+"_"+str(c)+"_mask.tif"),split_mask[c])

In [24]:
help(io.load_train_test_data)

Help on function load_train_test_data in module cellpose.io:

load_train_test_data(train_dir, test_dir=None, image_filter=None, mask_filter='_masks', look_one_level_down=False)
    Loads training and testing data for a Cellpose model.
    
    Args:
        train_dir (str): The directory path containing the training data.
        test_dir (str, optional): The directory path containing the testing data. Defaults to None.
        image_filter (str, optional): The filter for selecting image files. Defaults to None.
        mask_filter (str, optional): The filter for selecting mask files. Defaults to "_masks".
        look_one_level_down (bool, optional): Whether to look for data in subdirectories of train_dir and test_dir. Defaults to False.
    
    Returns:
        images, labels, image_names, test_images, test_labels, test_image_names



In [ ]:
train_dir = 'E:\ParadaKusz_Lab\Microglia_CellPose_Model\Training\'
test_dir = 'E:\ParadaKusz_Lab\Microglia_CellPose_Model\Test\'
io.logger_setup()

output = io.load_train_test_data(train_dir, test_dir,
                                mask_filter="_mask", look_one_level_down=False)
images, labels, image_names, test_images, test_labels, image_names_test = output

model = models.CellposeModel(model_type="cyto3",gpu=True)

model_path, train_losses, test_losses = train.train_seg(model.net,
                            train_data=images, train_labels=labels,
                            test_data=test_images, test_labels=test_labels,
                            weight_decay=0.1, learning_rate=1e-5,
                            n_epochs=100, model_name="Test")

41